## This is for the exam project


In [1]:
##Imports
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup as soup
import bs4
import requests
import re
import sys

### Numberplate to string without img handling

In [2]:
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/4.1.1/bin/tesseract'

img = cv2.imread("./images/numberplate3.png")
text = pytesseract.image_to_string(img)
print(text)


BW 80 777

 



### Numberplate to string with img handling

In [6]:
import cv2
import imutils
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

img2 = cv2.imread('./images/numberplate3.png',cv2.IMREAD_COLOR)
#img2 = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 

edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
#new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("Nummerplade: ",text)
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

TesseractNotFoundError: C:\Program Files\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.

### Entering numberplate on nummerplade.net to find model

In [6]:
print("Starting up..")
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
options = Options()
options.headless = True

# Lav jeres egen driver! 
browser = webdriver.Chrome(executable_path="/Users/frederikdahl/chromedriver/chromedriver")

browser.get("https://www.nummerplade.net/")

print("Got Browser")

browser.implicitly_wait(3)

#cookie_button = browser.find_element_by_css_selector('div.coi-banner__page-footer:nth-child(2) > button:nth-child(2)')
#cookie_button.click()
#print("Cookie Button Clicked..")
#sleep(3) 

search_field = browser.find_element_by_id('search_regnr')

search_field.send_keys(text)

button = browser.find_element_by_id('search_regnr_button')
button.click()
print("Search Entered")
sleep(3)

print()

page_source = browser.page_source
page_soup = soup(page_source,'html.parser')
res = page_soup.select_one("#maerke").text +" "+ page_soup.select_one("#model").text +" " + page_soup.select_one("#variant").text
print("Søgte bil: ", res)
browser.quit()

Starting up..
Got Browser..
Search Entered..

Søgte bil:  Volkswagen Golf 1.4 Tsi Bmt 122 Hk 4 Dørs


### Searching for model on bilbasen to get model,price,km, 

In [4]:
base_url = 'https://www.bilbasen.dk/'

class my_car:
    def __init__(self, url):
        self.base_url = url
        self.car_list = []
        
        print("Starting up...")
        
        profile = webdriver.FirefoxProfile()
        profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
        options = Options()
        options.headless = True
        
        self.browser = webdriver.Firefox(options=options)
        
        self.browser.get(self.base_url)
        print("Got Browser")
        
        self.browser.implicitly_wait(3)
        
    def click_cookie_button(self):
        try:
            cookie_button = self.browser.find_element_by_id('onetrust-accept-btn-handler')
            try:
                cookie_button.click()
                print("Cookie Button Clicked")
                print()
                sleep(3)   
            except Exception as err:
                print('Error With Cookie Button:', err)
        except Exception as err:
            print(err)
            
    def click_leasing_button(self):
        try:
            leasing_button = self.browser.find_element_by_css_selector("label[data-track-action='leasing-toggle']")
            try:
                leasing_button.click()
                print("Leasing Button Clicked")
                print()
                sleep(3)   
            except Exception as err:
                print('Error With Leasing Button:', err)
        except Exception as err:
            print(err)
        
    def search_in_search_bar(self, search_text):
        try:
            search_field = self.browser.find_element_by_class_name('react-autosuggest__input')
            search_field.send_keys(search_text)
            search_field.submit()
            print("Search Bar Entered")
            print()
            sleep(3)

        except Exception as err:
            print('Error with Search', err)

    def get_car_list(self):
        print("Souping Site...")
        print()
        page_source = self.browser.page_source
        page_soup = soup(page_source,'html.parser')
        car_list_plus = page_soup.findAll("div",{"class":"row listing listing-plus bb-listing-clickable"})
        car_list_discount = page_soup.findAll("div",{"class":"row listing listing-discount bb-listing-clickable"})
        full_list = car_list_plus + car_list_discount
        print("Cars found: " + str(len(full_list)))
        for car in full_list:
            name = car.find("a",{"class":"listing-heading darkLink"}).contents[0]
            km = int(car.findAll("div",{"class":"col-xs-2 listing-data"})[1].contents[0].replace('.',''))
            price = car.find("div",{"class":"col-xs-3 listing-price"}).contents[0].replace(' kr.','').replace('.','')
            car_info = {"name": name, "km": km, "price": price}
            self.car_list.append(car_info)
            print(car_info) 
            
    def close_browser(self):
        self.browser.close()
    
bilbasen = my_car(base_url)
bilbasen.click_cookie_button()
bilbasen.click_leasing_button()
bilbasen.search_in_search_bar("VW POLO")
bilbasen.get_car_list()
bilbasen.close_browser()

Starting up...
Got Browser

Cookie Button Clicked

Leasing Button Clicked

Search Bar Entered

Souping Site...

Cars found: 31
{'name': 'VW Polo 1,0 TSi 95 Comfortline DSG 5d', 'km': 43000, 'price': '184980'}
{'name': 'VW Polo 1,0 TSi 115 Highline DSG 5d', 'km': 69000, 'price': '179999'}
{'name': 'VW Polo 1,0 TSi 95 Comfortline 5d', 'km': 0, 'price': '208900'}
{'name': 'VW Polo 1,5 TSi 150 Comfortline DSG 5d', 'km': 6000, 'price': '229900'}
{'name': 'VW Polo 1,0 TSi 115 Highline DSG 5d', 'km': 31000, 'price': '214900'}
{'name': 'VW Polo 2,0 GTi DSG 5d', 'km': 12000, 'price': '279900'}
{'name': 'VW Polo 1,4  5d', 'km': 115000, 'price': '24990'}
{'name': 'VW Polo 1,2 TSi 90 Comfortline DSG BMT 5d', 'km': 88000, 'price': '169900'}
{'name': 'VW Polo 1,2 TDi 75 BlueMotion 5d', 'km': 161000, 'price': '74990'}
{'name': 'VW Polo 1,4 Comfortline 5d', 'km': 132000, 'price': '74990'}
{'name': 'VW Polo 1,0 TSi 115 Highline DSG 5d', 'km': 9000, 'price': '219900'}
{'name': 'VW Polo 1,0 TSi 95 BlueMo